# Mesh generation for tutorial 06, case 7b

This file generates the mesh which is used in the following examples:
* 7b_stokes_neumann_control

The test case is from section 5.5 of
F. Negri. Reduced basis method for parametrized optimal control problems governed by PDEs. Master thesis, Politecnico di Milano, 2010-2011.

In [ ]:
import numpy as np
import pygmsh
from mpi4py import MPI
from dolfinx import cpp
from dolfinx.mesh import create_mesh, create_meshtags
from dolfinx.io import ufl_mesh_from_gmsh, XDMFFile
from dolfinx.plotting import plot

In [ ]:
assert MPI.COMM_WORLD.size == 1, "This mesh generation notebook is supposed to be run in serial"

### Geometrical parameters and related quantities

In [ ]:
mu1 = 1.0
mu2 = np.pi / 5.0
mu3 = np.pi / 6.0
mu4 = 1.0
mu5 = 1.7
mu6 = 2.2
lcar = 0.05

In [ ]:
Y = 1.0
X = -Y
L = 3.0
B = Y - mu1
H_1 = B + np.tan(mu2) * mu5
H_2 = B - np.tan(mu3) * mu6
L_1 = mu1 * np.cos(mu2) * np.sin(mu2)
L_2 = (B - X) * np.cos(mu3) * np.sin(mu3)
N = mu1 * np.cos(mu2) * np.cos(mu2)
M = - (B - X) * np.cos(mu3) * np.cos(mu3)

### Create pygmsh mesh

In [ ]:
geom = pygmsh.geo.Geometry()
p0 = geom.add_point([0.0, X, 0.0], lcar)
p1 = geom.add_point([L - mu4, X, 0.0], lcar)
p2 = geom.add_point([L, X, 0.0], lcar)
p3 = geom.add_point([L + mu6 - L_2, H_2 + M, 0.0], lcar)
p4 = geom.add_point([L + mu6, H_2, 0.0], lcar)
p5 = geom.add_point([L, B, 0.0], lcar)
p6 = geom.add_point([L + mu5, H_1, 0.0], lcar)
p7 = geom.add_point([L + mu5 - L_1, H_1 + N, 0.0], lcar)
p8 = geom.add_point([L, Y, 0.0], lcar)
p9 = geom.add_point([L - mu4, Y, 0.0], lcar)
p10 = geom.add_point([0.0, Y, 0.0], lcar)
l0 = geom.add_line(p0, p1)
l1 = geom.add_line(p1, p2)
l2 = geom.add_line(p2, p3)
l3 = geom.add_line(p3, p4)
l4 = geom.add_line(p4, p5)
l5 = geom.add_line(p5, p6)
l6 = geom.add_line(p6, p7)
l7 = geom.add_line(p7, p8)
l8 = geom.add_line(p8, p9)
l9 = geom.add_line(p9, p10)
l10 = geom.add_line(p10, p0)
l11 = geom.add_line(p1, p9)
l12 = geom.add_line(p2, p5)
l13 = geom.add_line(p5, p8)
geom.add_physical([l10], label=1)
geom.add_physical([l0, l1, l2, l4, l5, l7, l8, l9], label=2)
geom.add_physical([l3, l6], label=3)
geom.add_physical(l11, label=4)
line_loop_rectangle_left = geom.add_line_loop([l0, l11, l9, l10])
line_loop_rectangle_right = geom.add_line_loop([l1, l12, l13, l8, -l11])
line_loop_bifurcation_top = geom.add_line_loop([l5, l6, l7, -l13])
line_loop_bifurcation_bottom = geom.add_line_loop([l2, l3, l4, -l12])
rectangle_left = geom.add_plane_surface(line_loop_rectangle_left)
rectangle_right = geom.add_plane_surface(line_loop_rectangle_right)
bifurcation_top = geom.add_plane_surface(line_loop_bifurcation_top)
bifurcation_bottom = geom.add_plane_surface(line_loop_bifurcation_bottom)
geom.add_physical(rectangle_left, label=11)
geom.add_physical(rectangle_right, label=12)
geom.add_physical(bifurcation_top, label=13)
geom.add_physical(bifurcation_bottom, label=14)
pygmsh_mesh = pygmsh.generate_mesh(geom)

### Convert to a dolfinx mesh

In [ ]:
cells, x = pygmsh_mesh.cells_dict["triangle"], pygmsh_mesh.points[:, :2]
mesh = create_mesh(MPI.COMM_WORLD, cells, x, ufl_mesh_from_gmsh("triangle", x.shape[1]))

In [ ]:
plot(mesh)

### Extract subdomains as MeshTags

In [ ]:
subdomains_entities = cells
subdomains_values = pygmsh_mesh.cell_data_dict["gmsh:physical"]["triangle"] - 10
subdomains = create_meshtags(mesh, mesh.topology.dim,
                             cpp.graph.AdjacencyList_int32(subdomains_entities), np.int32(subdomains_values))
subdomains.name = "subdomains"

### Extract boundaries as MeshTags

In [ ]:
boundaries_entities = pygmsh_mesh.cells_dict["line"]
boundaries_values = pygmsh_mesh.cell_data_dict["gmsh:physical"]["line"]
mesh.topology.create_connectivity(mesh.topology.dim - 1, 0)
boundaries = create_meshtags(mesh, mesh.topology.dim - 1,
                             cpp.graph.AdjacencyList_int32(boundaries_entities), np.int32(boundaries_values))
boundaries.name = "boundaries"

### Save final mesh, subdomains and boundaries (dolfinx format)

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "bifurcation.xdmf", "w") as file:
    file.write_mesh(mesh)
    file.write_meshtags(subdomains)
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    file.write_meshtags(boundaries)